# Kicker Web Scraping

This notebook scrapes all matchday tables from the 2021-22 Bundesliga season from __[Kicker](https://www.kicker.de/)__ using BeautifulSoup.
The data is then cleaned and enriched by a column containing a link for each team's icon. The final data is saved to csv.

Using the scraped data, a Tableau dashboard has been created showing a team's position on each of the 34 matchdays. The dashboard can be found on __[Tableau Public](https://public.tableau.com/app/profile/sarah.houy/viz/BundesligaSeason2122Review/Season_Review_Overview#1)__.


In [ ]:
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import numpy as np
import buli_icons_config

### Scraping relevant data

In [ ]:
df_tables = None # will contain data of all 34 matchday tables

# Loop through every matchday
for matchday in range(1, 35, 1):
  # Get the "table" HTML elements that exist on the page:  
  scrapper = pd.read_html('https://www.kicker.de/bundesliga/tabelle/2021-22/' + str(matchday))
  
  # The 0 position is the one that contains the desired data:       
  for idx, table in enumerate(scrapper):
    if (idx == 0): 
      # Remove certain columns that are not needed: 
      table = table.drop(table.columns[[1, 2, 5, 6, 7, 9]], axis=1)

      if (df_tables is None): 
        df_tables = table
      else: 
        df_tables = df_tables.append(table, ignore_index=True)

### Data Cleaning

In [ ]:
# Keep only the full team name
team_cleaned = df_tables["Team"].str.split("  ", n = 1, expand = True)

# Replace team in df by team_cleaned
df_tables["Team"]= team_cleaned[1]

# Split goals in goals scored and goals conceded and add them to df
goals_split = df_tables["Tore"].str.split(":", n = 1, expand = True)
df_tables["Tore_erzielt"] = goals_split[0]
df_tables["Tore_kassiert"] = goals_split[1]

# When teams have the same position on a matchday the column Pl. is NaN
# NaN is replaced by the last number preceding the NaN value
df_tables.fillna(method='ffill', inplace=True)
# convert Pl. column to int
df_tables["Pl."] = df_tables["Pl."].astype('int64')

### Enriching dataframe

In [ ]:
# define variable that contains links for logo of each team
# this column is used to display team logo in Tableau
links = ['https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/VfB_Stuttgart_1893_Logo.svg/500px-VfB_Stuttgart_1893_Logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Logo_TSG_Hoffenheim.svg/500px-Logo_TSG_Hoffenheim.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Borussia_Dortmund_logo.svg/500px-Borussia_Dortmund_logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Emblem_1.FC_K%C3%B6ln.svg/500px-Emblem_1.FC_K%C3%B6ln.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d6/FSV_Mainz_05_Logo.png/768px-FSV_Mainz_05_Logo.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Logo-VfL-Wolfsburg.svg/500px-Logo-VfL-Wolfsburg.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/1._FC_Union_Berlin_Logo.svg/500px-1._FC_Union_Berlin_Logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/de/thumb/f/f7/Bayer_Leverkusen_Logo.svg/500px-Bayer_Leverkusen_Logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Borussia_M%C3%B6nchengladbach_logo.svg/500px-Borussia_M%C3%B6nchengladbach_logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Logo_FC_Bayern_M%C3%BCnchen_%282002%E2%80%932017%29.svg/500px-Logo_FC_Bayern_M%C3%BCnchen_%282002%E2%80%932017%29.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Arminia-wappen-2021.svg/480px-Arminia-wappen-2021.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/SC_Freiburg_Wappen.svg/500px-SC_Freiburg_Wappen.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/VfL_Bochum_logo.svg/500px-VfL_Bochum_logo.png',
        'https://vectorportal.com/storage/rasen_ballsport_leipzig2.jpg',
        'https://upload.wikimedia.org/wikipedia/de/thumb/3/38/Hertha_BSC_Logo.svg/500px-Hertha_BSC_Logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Eintracht_Frankfurt_Logo.svg/500px-Eintracht_Frankfurt_Logo.svg.png',
        'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/SpVgg_Greuther_F%C3%BCrth_2017.svg/500px-SpVgg_Greuther_F%C3%BCrth_2017.svg.png',
        'https://upload.wikimedia.org/wikipedia/de/thumb/b/b5/Logo_FC_Augsburg.svg/500px-Logo_FC_Augsburg.svg.png']

In [ ]:
# create dictionary that contains logo link for every team
iconMapper = dict()
for link, team_name in enumerate(df_tables.Team.unique()):
    iconMapper[team_name] = links[link]    

# add corresponding link to each team
orderedLinks = []
for team_name in df_tables.Team:
    orderedLinks.append(iconMapper[team_name])

# add column "Icons" to df
df_tables['Icons'] = orderedLinks

### Saving data as csv

In [ ]:
# define current and target working directory and filename
cwd = Path('.').absolute()
file_name = 'bundesliga_21_22_with_icons.csv'
targetDir = Path(cwd, 'data')

In [ ]:
# create target directory
targetDir.mkdir(parents=True, exist_ok=True)

In [ ]:
# write data to csv
df_tables.to_csv(Path(targetDir, file_name))  